# Using Python with Earth Science Datasets: Cartopy, xarray, and pyresample

## Objective: working with Earth science datasets

* You will learn to:
	* Plot data on a map
	* Read netCDF datasets
	* Perform re-gridding, averaging, filtering, and out-of-memory operations
    
---

# Recap on packages
Packages give us additional functionality, saving us the trouble of writing procedures ourselves. 

Primary libraries from the last part...
* [NumPy](http://www.numpy.org/): Fast mathematical operations on arrays
* [Pandas](https://pandas.pydata.org): Operations and easy read/write of tabular data. Builds extra functionality on top of NumPy.
* [Matplotlib](https://matplotlib.org): Primary Python plotting/visualization package. You can generate line plots, histograms, scatter plots, etc., with just a few lines of code.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Bring back our list of aeronet stations...
filename = 'data/aeronet_locations_v3.txt'
station_list = pd.read_csv(filename, skiprows=1)
station_list.columns = ['site', 'lon', 'lat', 'elev']

## [Cartopy](https://scitools.org.uk/cartopy/)
* Cartopy is not included in Anaconda, need to install yourself.

Open the terminal (Mac/Linux) or Anaconda Prompt (Windows) and type:
```python
conda install -c conda-forge cartopy
```
* Rather than import all of Cartopy, we just want the projection classes to pair with matplotlib.
* More [map projections](https://scitools.org.uk/cartopy/docs/latest/crs/projections.html).

In [ ]:
# Options to increase figure size
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams.update({'font.size': 16})

from cartopy import crs as ccrs

In [ ]:
from_proj = ccrs.PlateCarree()
to_proj = ccrs.PlateCarree()

# Center on the Atlantic
ax = plt.axes(projection=to_proj)
ax.coastlines()
plt.scatter(station_list['lon'], station_list['lat'], color='blue', s=1, transform=from_proj)
plt.show()

<div class="alert alert-block alert-info">

**Exercise 1**

* Import aeronet_locations_v3.txt with read_csv
* Define the map axes using with option projection=ccrs.PlateCarree()
* Add coastlines
* Make a scatter plot of the station locations

* Challenge: Change the projection to Equal Earth and shift the plot to center on -180 using ccrs.EqualEarth(central_longitude=-180)
</div>

In [ ]:
# Solution: Center on the Pacific
from_proj = ccrs.PlateCarree()
to_proj = ccrs.EqualEarth(central_longitude=-180)

ax = plt.axes(projection=to_proj)
ax.coastlines()
plt.scatter(station_list['lon'], station_list['lat'], color='red', s=1, transform=from_proj)
plt.show()

## Examining the 2018 California Wildfires from Space

* 6,870 fires had burned over a 6,000 km${^2}$ area. 
* The smoke from the wildfires also had an impact on air quality both in proximity of the fires as well as across the country.
* We'll look at satellite observations from __Suomi-NPP__ and __GOES-17__ to show the impact of the California wildfires on AOD.

![](figures/satellites.png)

We're in the golden age of sallite datasets, which is a blessing and a curse:

* Inundated with datasets, don't know which ones to use
* No single repository for access of the data
* Inconsistent formatting and filetypes

netCDF4 and HDF5 are the dominant formats used in satellite remote sensing (but others do exist)

## [xarray](https://xarray.pydata.org/en/stable/)

* xarray is like pandas, but for N-dimensional arrays.
* Can read and write NetCDF, HDF5, and GRIB (with the additional cfgrib package) files.
* Can perform efficient operations on these datasets, including out-of-memory and parallel operations.

### LEO Example: Suomi-NPP
LEO Satellites orbit the Earth many times a day, data are organized in 1 minute swaths per file. We'll be looking at aerosol optical depth data over California.

In [ ]:
import xarray

In [ ]:
# Import your data...
filename = 'data/JRR-AOD_v1r2_npp_s201811082130296_e201811082131537_c201811082228260.nc'
npp = xarray.open_dataset(filename)

# Print a list of variables
list(npp.variables)

In [ ]:
AOD_NPP = npp['AOD550']
lat_NPP = npp['Latitude']
lon_NPP = npp['Longitude']

In [ ]:
# xarray has automatically read in the missing values as nan
AOD_NPP

In [ ]:
AOD_NPP.mean()

In [ ]:
plt.pcolormesh(lon_NPP, lat_NPP, AOD_NPP)
plt.colorbar()
plt.show()

In the next plot, we'll
* Add in the cartopy map
* Change the color scheme
    * See the [matplotlib color schemes](https://matplotlib.org/examples/color/colormaps_reference.html) and [ColorBrewer schemes](http://colorbrewer2.org).
    * Recommend [cmocean](https://matplotlib.org/cmocean/), a collection of colormaps for oceanography but great for other purposes too.
* There is an option to zoom in on the data using plt.xlim and plt.ylim

In [ ]:
from_proj = ccrs.PlateCarree()
to_proj = ccrs.PlateCarree()

levs = np.arange(0, 4, 0.1)

# Using cartopy, create the map projection and plot the data
ax = plt.axes(projection=to_proj)
ax.coastlines()

# Can change color scheme using the cmap argument
plt.contourf(lon_NPP, lat_NPP, AOD_NPP, levs, transform=from_proj, extend='max',
             cmap="PuBu")

plt.colorbar(orientation="horizontal", fraction=0.07)

plt.xlim(-145, -110)
plt.ylim(30, 50)

# To zoom in on data:
#plt.xlim(-125, -120)
#plt.ylim(38, 44)
plt.show()

<div class="alert alert-block alert-info">

**Exercise 2** 

??? = Fill in the blanks!

**Import from netCDF files**

* Import JRR-AOD_v1r1_npp_s201808091957192_e201808091958434_c201808092051240.nc using the xarray.open_dataset command.
* Inpect the available variables.
* Save the latitude, longitude, and AOD to arrays.

**Create a cartopy plot**

* Define the axes, including the projection. 
    * Challenge: Make an orthographic plot using: ccrs.Orthographic(central_longitude=-75.0, central_latitude=0.0)
* Create a plot using plt.contourf(???, ???, ???, transform=???)
    * Challenge: Change the data scale from the default to 0.0-1.6
* Remember to plt.show() at the end to display!

</div>

## Common tasks
1. Regridding
2. Masking datasets
3. Filtering with Quality Flags

### Regridding

Regridding is the process of interpolating from one grid resolution to a different grid resolution and [UCAR's website has a good discussion](https://climatedataguide.ucar.edu/climate-data-tools-and-analysis/regridding-overview). 

There are a few options:

* Interpolate using griddata from SciPy package (2D interpolate too slow/limited number of points)
* Regridding in [iris](https://scitools.org.uk/iris/docs/latest/) package (autodetection of GRIB and NC fileformats... if they follow the conventions!)
* [xESMF](https://xesmf.readthedocs.io/en/latest/), a Python package for regridding for gridded datasets (not designed for satellite data)
* __Pyresample__: the syntax can be fairly complex, but very fast

Pyresample also is not included in Anaconda, so install using:

```
conda install -c conda-forge pyresample

```

Steps:
1. Define the new grid (either in Python or import an irregular array)
2. Define the swath/satellite grid
3. Resample to get the new values for the updated grid

In [ ]:
from pyresample import geometry
from pyresample.kd_tree import resample_nearest

In [ ]:
# Create a new grid at 0.1 degree resolution
x = np.arange(lon_NPP.min(), lon_NPP.max(), 0.1)
y = np.arange(lat_NPP.min(), lat_NPP.max(), 0.1)
new_lon, new_lat = np.meshgrid(x,y)

# define the new grid using 
new_lonlat = geometry.GridDefinition(lons=new_lon, lats=new_lat)

In [ ]:
# Input list of swath points
old_lonlat = geometry.SwathDefinition(lons=lon_NPP, lats=lat_NPP)

# Resample the data
new_AOD = resample_nearest(old_lonlat, AOD_NPP.values, new_lonlat, radius_of_influence=5000, fill_value=None)

In [ ]:
# To compare: Putting two plots together using plt.subplot
plt.subplot(1, 2, 1)
plt.title("Before regridding")
plt.pcolormesh(lon_NPP, lat_NPP, AOD_NPP, vmin=0, vmax=1)

plt.subplot(1, 2, 2)
plt.pcolormesh(new_lon, new_lat, new_AOD, vmin=0, vmax=1)
plt.title("After regridding")
plt.show()

### Masking datasets

May want to add a land/ocean mask to our datasets

In [ ]:
import cartopy.feature as feature

In [ ]:
from_proj = ccrs.PlateCarree()
to_proj = ccrs.PlateCarree()

ax = plt.axes(projection=to_proj)
ax.coastlines()
plot = ax.contourf(lon_NPP, lat_NPP, AOD_NPP, transform=from_proj)

# Mask out the ocean: zorder places it on top of the existing data
ax.add_feature(feature.OCEAN, zorder=100, edgecolor='black')

cb = plt.colorbar(plot, orientation="horizontal", fraction=0.07)
plt.show()

<div class="alert alert-block alert-info">

**Exercise 3**

* Filter your plot in Exercise 2 (using NPP data) by data quality using the variable 'QCAll'
* Import QCAll into a variable
* Create a mask for medium/low quality data or pixels with no retrieval
* Make a new map with the filtered data

</div>

In [ ]:
qc_flag = npp['QCAll']

mask_HQ = (qc_flag != 0)
AOD_NPP_HQ = np.ma.masked_where(mask_HQ, qc_flag)

## Saving images

* Create the plot, then use plt.savefig

In [ ]:
from_proj = ccrs.PlateCarree()
to_proj = ccrs.PlateCarree()

levs = np.arange(0, 1.8, 0.1)

ax = plt.axes(projection=to_proj)
plt.contourf(lon_NPP, lat_NPP, AOD_NPP_HQ, levs, transform=from_proj)
ax.coastlines()
plt.colorbar(orientation="horizontal", fraction=0.07)

# Add DPI option to change resolution of the plot (maintains the same aspect ratio)
plt.savefig('Wildfires_AOD_2018221_1612.png', dpi=500)

# TIP: If you want to suppress the output (runs faster!), use the following:
plt.close()

## Out-of-memory processing

* Some datasets are so big that it is not a good idea to load them all into memory.
* xarray operations can be run on successive pieces of datasets so the entire data is never in memory. Operations will be run in parallel! [See more details.](https://xarray.pydata.org/en/stable/dask.html)
* We'll use the MERRA-2 reanalysis of AOD.

In [ ]:
filename = 'data/MERRA2_400.inst3_2d_gas_Nx.20181109.nc4.nc'
merra = xarray.open_dataset(filename, chunks={'time': 1})

In [ ]:
merra['AODANA']

In [ ]:
# Let's do a time-mean

merra['AODANA'].mean(dim='time')

The mean does not compute! xarray will avoid doing the computation with out-of-memory datasets until you tell it to.

In [ ]:
mean_AOD = merra['AODANA'].mean(dim='time').compute()
mean_AOD

In [ ]:
from_proj = ccrs.PlateCarree()
to_proj = ccrs.EqualEarth()

# Using cartopy, create the map projection and plot the data
ax = plt.axes(projection=to_proj)
ax.coastlines()

# Can change color scheme using the cmap argument
plt.pcolormesh(merra['lon'], merra['lat'], mean_AOD, transform=from_proj)
plt.colorbar(label='AOD')

plt.show()

## Scripting

* Notebooks are nice for sharing results with others, but scripts are useful for automating tasks.
* You can export a script from Jupyter Notebook directly: File &rarr; Download As &rarr; Python (.py)
* To make a script, write code into a [filename].py file with the following shebang:
```python
#!/usr/bin/env python
```

For example, let's make the file `hello_world.py`:

```python
#!/usr/bin/env python

print("Hello world!")
```

Then to call the script, open the terminal, check the file permissions, and simply type:
```python
python hello_world.py
```
Windows users: you'll need to run the scripts through the Anaconda Prompt and use the full path to the script using double backslashes (\\\\). You may need to associate \*.py files with python (C:\Users\[your username]\Anaconda3\python) 


## Resources
* [PyTroll](http://pytroll.github.io) framework for the processing of Earth observation satellite data
* [Unidata AWIPS Python tutorial](http://unidata.github.io/awips2/python/satellite-imagery/)
* [Workshop on developing Python frameworks for earth system sciences](https://www.ecmwf.int/en/learning/workshops/2018-workshop-developing-python-frameworks-earth-system-sciences)
* [Python Satellite Data Analysis Toolkit (pysat)](https://github.com/rstoneback) Toolkit for space science data analysis.